# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(5):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    step = 0
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        step += 1
        if np.any(dones):                                  # exit loop if episode finished
            print(step)
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

36
Score (max over agents) from episode 0: 0.0
15
Score (max over agents) from episode 1: 0.0
33
Score (max over agents) from episode 2: 0.09000000171363354
14
Score (max over agents) from episode 3: 0.0
14
Score (max over agents) from episode 4: 0.0


In [7]:
from maddpg import MADDPG
import torch
maddpg = MADDPG(state_size, action_size, num_agents, discount_factor=0.95, tau=0.02)
for i_episode in range(10):
        env_info = env.reset(train_mode=False)[brain_name]
        states = env_info.vector_observations
        while True:
            actions = maddpg.act(torch.tensor(states, dtype=torch.float))
            print(actions)
            #actions_array = torch.stack(actions).detach().numpy()
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            states = next_states
            dones = env_info.local_done
            if np.any(dones):
                break


[array([-0.03236611,  0.0935538 ], dtype=float32), array([ 0.19367296, -0.00614072], dtype=float32)]
[array([0.13373776, 0.27176756], dtype=float32), array([-0.09918828, -0.08457644], dtype=float32)]
[array([-0.11300151,  0.26312262], dtype=float32), array([0.24050136, 0.2018709 ], dtype=float32)]
[array([0.00487806, 0.12772372], dtype=float32), array([ 0.0286371, -0.0598582], dtype=float32)]
[array([0.09455887, 0.5943183 ], dtype=float32), array([-0.34254757,  0.18530175], dtype=float32)]
[array([-0.13743864,  0.4589889 ], dtype=float32), array([0.04931681, 0.18910065], dtype=float32)]
[array([0.14418279, 0.70778966], dtype=float32), array([-0.20636612,  0.2572966 ], dtype=float32)]
[array([0.50696534, 0.64175475], dtype=float32), array([-0.10184778,  0.23080023], dtype=float32)]
[array([0.34913173, 0.5835305 ], dtype=float32), array([0.17486769, 0.18902558], dtype=float32)]
[array([0.5082835 , 0.34023803], dtype=float32), array([0.20163652, 0.84682965], dtype=float32)]
[array([0.4108

[array([-0.12040874, -0.12461653], dtype=float32), array([-0.30010182,  0.1503113 ], dtype=float32)]
[array([-0.18341392,  0.23973309], dtype=float32), array([0.1770927 , 0.18865302], dtype=float32)]
[array([0.18589503, 0.23700449], dtype=float32), array([0.24671432, 0.26643863], dtype=float32)]
[array([0.4590381 , 0.17145336], dtype=float32), array([0.46877488, 0.21378149], dtype=float32)]
[array([0.23418756, 0.31115195], dtype=float32), array([ 0.1160346, -0.0201789], dtype=float32)]
[array([-0.10768832,  0.23761603], dtype=float32), array([-0.17601077, -0.42650324], dtype=float32)]
[array([-0.02993429,  0.19534728], dtype=float32), array([-0.38067394, -0.11287855], dtype=float32)]
[array([0.16092163, 0.6175034 ], dtype=float32), array([-0.20388821, -0.25297338], dtype=float32)]
[array([0.0097699 , 0.42760307], dtype=float32), array([ 0.18235934, -0.07173216], dtype=float32)]
[array([-0.00695729,  0.4142195 ], dtype=float32), array([-0.22755003, -0.12468968], dtype=float32)]
[array([

#### When finished, you can close the environment.

In [7]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```